In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf

# Load models
cnn_model = tf.keras.models.load_model(r"D:\minor\test_k_fold_CNN_landmark\model_fold_5.keras")
cnn_lstm_model = tf.keras.models.load_model(r"D:\minor\k_fold_CNN_LSTM_landmark\model_fold_3.keras")

# Load test data
X_test = np.load(r"D:\minor\workout_processed_data_landmark\X.npy")
y_test = np.load(r"D:\minor\workout_processed_data_landmark\y.npy")
class_mapping = np.load(r"D:\minor\workout_processed_data_landmark\class_mapping.npy", allow_pickle=True).item()

# Reverse class mapping to {index: class_name}
class_mapping_reversed = {v: k for k, v in class_mapping.items()}

# Generate predictions
cnn_predictions = cnn_model.predict(X_test)
cnn_lstm_predictions = cnn_lstm_model.predict(X_test)

# Convert probabilities to class labels
cnn_predictions_labels = np.argmax(cnn_predictions, axis=1)
cnn_lstm_predictions_labels = np.argmax(cnn_lstm_predictions, axis=1)

# Debug: Print unmapped labels
unmapped_labels = set(np.unique(cnn_predictions_labels)) - set(class_mapping_reversed.keys())
if unmapped_labels:
    print("Warning: Unmapped Labels in CNN Predictions ->", unmapped_labels)

unmapped_labels_lstm = set(np.unique(cnn_lstm_predictions_labels)) - set(class_mapping_reversed.keys())
if unmapped_labels_lstm:
    print("Warning: Unmapped Labels in CNN-LSTM Predictions ->", unmapped_labels_lstm)

# Map predictions to class names safely
cnn_predictions_classes = [class_mapping_reversed.get(label, "Unknown") for label in cnn_predictions_labels]
cnn_lstm_predictions_classes = [class_mapping_reversed.get(label, "Unknown") for label in cnn_lstm_predictions_labels]
y_test_classes = [class_mapping_reversed.get(label, "Unknown") for label in y_test]

# Store in DataFrame
df = pd.DataFrame({
    "frame": np.arange(1, len(y_test) + 1),
    "ground_truth": y_test_classes,
    "cnn_prediction": cnn_predictions_classes,
    "cnn_lstm_prediction": cnn_lstm_predictions_classes,
    "cnn_confidence": cnn_predictions.max(axis=1),
    "cnn_lstm_confidence": cnn_lstm_predictions.max(axis=1)
})

# Save to CSV
df.to_csv("predictions.csv", index=False)
print("Predictions saved to predictions.csv")


17/17 [==============================] - 2s 48ms/step
Predictions saved to predictions.csv


In [2]:
print("Class Mapping Keys:", class_mapping.keys())
print("Example CNN Predicted Labels:", np.unique(cnn_predictions_labels))
print("Example CNN-LSTM Predicted Labels:", np.unique(cnn_lstm_predictions_labels))


Class Mapping Keys: dict_keys(['barbell biceps curl', 'hammer curl', 'lat pulldown', 'lateral raise', 'pull Up', 'push-up', 'shoulder press'])
Example CNN Predicted Labels: [0 1 2 3 4 5 6]
Example CNN-LSTM Predicted Labels: [0 1 2 3 4 5 6]


In [3]:
unmapped_labels = set(np.unique(cnn_predictions_labels)) - set(class_mapping.keys())
print("Unmapped Labels:", unmapped_labels)


Unmapped Labels: {0, 1, 2, 3, 4, 5, 6}


In [4]:
cnn_predictions_classes = [class_mapping.get(label, "Unknown") for label in cnn_predictions_labels]
cnn_lstm_predictions_classes = [class_mapping.get(label, "Unknown") for label in cnn_lstm_predictions_labels]
y_test_classes = [class_mapping.get(label, "Unknown") for label in y_test]
